## 1. Preprocesamiento de Datos

### Conversión de variables categóricas:
- Convertir directamente las variables categóricas a numéricas.
- Utilizar **Label Encoding** o **OneHotEncoding**.
- Representar las variables categóricas utilizando la probabilidad de la clase objetivo.

### Imputación de valores faltantes:
- Imputar los valores faltantes utilizando el valor **0**.
- Imputar los valores faltantes utilizando la **media** para variables numéricas.


In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer

In [27]:
# Cargar los datos
train_path = '/home/albaro/Documentos/DATA/titanic/train.csv'
test_path = '/home/albaro/Documentos/DATA/titanic/test.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# Concatenar los datasets para preprocesarlos juntos
data = pd.concat([train, test], sort=False)

# Eliminar la columna 'Cabin' para evitar problemas por valores nulos
data.drop(columns=['Cabin'], inplace=True)

# ======== IMPUTACIÓN DE VALORES FALTANTES ========

# Imputar variables numéricas con la media
num_cols = data.select_dtypes(include=['int64', 'float64']).columns
num_imputer = SimpleImputer(strategy='mean')
data[num_cols] = num_imputer.fit_transform(data[num_cols])

# Imputar variables categóricas con 'missing'
cat_cols = data.select_dtypes(include='object').columns
data[cat_cols] = data[cat_cols].fillna('missing')

# ======== CONVERSIÓN DE VARIABLES CATEGÓRICAS ========

# Aplicar OneHotEncoding a las variables categóricas seleccionadas
cols_to_encode = ['Sex', 'Embarked', 'Pclass']
data = pd.get_dummies(data, columns=cols_to_encode)

print(data.info())
data

PassengerId         0
Survived            0
Name                0
Age                 0
SibSp               0
Parch               0
Ticket              0
Fare                0
Sex_female          0
Sex_male            0
Embarked_C          0
Embarked_Q          0
Embarked_S          0
Embarked_missing    0
Pclass_1.0          0
Pclass_2.0          0
Pclass_3.0          0
dtype: int64


,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_missing,Pclass_1.0,Pclass_2.0,Pclass_3.0
0,1.0,0.000000,"Braund, Mr. Owen Harris",22.000000,1.0,0.0,A/5 21171,7.2500,False,True,False,False,True,False,False,False,True
1,2.0,1.000000,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.000000,1.0,0.0,PC 17599,71.2833,True,False,True,False,False,False,True,False,False
2,3.0,1.000000,"Heikkinen, Miss. Laina",26.000000,0.0,0.0,STON/O2. 3101282,7.9250,True,False,False,False,True,False,False,False,True
3,4.0,1.000000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.000000,1.0,0.0,113803,53.1000,True,False,False,False,True,False,True,False,False
4,5.0,0.000000,"Allen, Mr. William Henry",35.000000,0.0,0.0,373450,8.0500,False,True,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305.0,0.383838,"Spector, Mr. Woolf",29.881138,0.0,0.0,A.5. 3236,8.0500,False,True,False,False,True,False,False,False,True
414,1306.0,0.383838,"Oliva y Ocana, Dona. Fermina",39.000000,0.0,0.0,PC 17758,108.9000,True,False,True,False,False,False,True,False,False
415,1307.0,0.383838,"Saether, Mr. Simon Sivertsen",38.500000,0.0,0.0,SOTON/O.Q. 3101262,7.2500,False,True,False,False,True,False,False,False,True
416,1308.0,0.383838,"Ware, Mr. Frederick",29.881138,0.0,0.0,359309,8.0500,False,True,False,False,True,False,False,False,True


In [28]:
# 1. Volver a separar train y test
train_preprocessed = data.loc[data['Survived'].notna()].copy()
test_preprocessed = data.loc[data['Survived'].isna()].copy()

# 2. Variables categóricas para codificar con target mean
target_encode_cols = ['Sex_missing', 'Sex_female', 'Sex_male',  # ya están one-hot encoded, así que mejor usar la original
                      'Sex', 'Embarked', 'Pclass']  # si no eliminaste las originales

# Revertimos a las categóricas originales si las tienes (si no, usa otras que aún no estén codificadas)
# Vamos a asumir que conservaste las columnas originales 'Sex', 'Embarked', y 'Pclass'
# Si las eliminaste al hacer OneHotEncoding, déjamelo saber y lo ajustamos

for col in ['Sex', 'Embarked', 'Pclass']:
    mean_map = train_preprocessed.groupby(col)['Survived'].mean()
    # Nueva columna con el nombre + '_target_enc'
    train_preprocessed[col + '_target_enc'] = train_preprocessed[col].map(mean_map)
    test_preprocessed[col + '_target_enc'] = test_preprocessed[col].map(mean_map)  # test usa el mapping de train


KeyError: 'Sex'

## 2. Selección de Características

Explorar diferentes métodos de selección de características:

- Filtrado hi-cuadrado.
- Embebido RFE.
- Envolvente regresión logística.


## 3. Evaluación de Modelos con Validación Cruzada
Evaluar para la combinación de cada una de las estrategias de codificación
de categorías, imputación y selección de características

- Implementar al menos **dos modelos predictivos**, por ejemplo:
  - **Random Forest**
  - **XGBoost**

- Utilizar **validación cruzada de 10 folds** para evaluar el rendimiento de cada modelo.

- Obtener las siguientes **métricas de desempeño**:
  - Precisión
  - Exactitud
  - Recall
  - F1-Score


## 4. Almacenamiento de Resultados en MySQL

Diseñar y crear una base de datos en **MySQL** para almacenar los resultados de las evaluaciones.  
Se recomienda incluir las siguientes tablas:

- **Información del preprocesamiento**: (estrategia de imputación, codificación utilizada, etc.).

- **Métricas de desempeño por modelo y por fold**  

- **Resumen general de la evaluación**: (por ejemplo, promedio y desviación estándar de las métricas).
